In [2]:
# import modules
import pandas as pd
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score, StratifiedKFold
import xgboost as xgb

df = pd.read_csv("TESS_simulateddata_combinedfeatures.csv", index_col=0)

X = df.drop(['Ids', 'CatalogY', 'ManuleY', 'CombinedY', 'Catalog_Period', 'Depth', 'Catalog_Epoch', 'SNR'], axis=1)

target = 'CombinedY'
predictors = [x for x in X.columns if x not in [target]]

print X.columns


def modelfit(alg, train, predictors, cv_folds=4):

    # StratifiedKFold for imbalanced data
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True)
    
    scores_roc = cross_val_score(alg, train[predictors], train['CombinedY'], cv=cv,
                                 scoring='roc_auc')
    
    scores_f1 = cross_val_score(alg, train[predictors], train['CombinedY'], cv=cv, 
                                scoring='f1')


    # Print model report:
    print "\nModel Report"
    print " Cross Validation roc_auc Scores : {0} ".format(scores_roc)
    print " Mean Score and Std. Dev. {0}, {1} ". format(scores_roc.mean(), 
                                                        scores_roc.std())
    print " Cross Validation f1 scores: {0} ".format(scores_f1)
    print "Cross Validation f1 scores mean and Std. Dev {0}, {1}".format(scores_f1.mean(), 
                                                                         scores_f1.std())


# initialize model and call fitting function
xgb1 = XGBClassifier(
    learning_rate=0.1,
    n_estimators=63,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    nthread=4,
    seed=27)

modelfit(xgb1, df, predictors)